# Shota Imanaga: Rookie to Sophomore Year Pitching Changes (2024-2025) | Statcast

## Introduction

**Shota Imanaga** is a Japanese left-handed pitcher who joined the **Chicago Cubs** in 2024 after a stellar career with the Yokohama DeNA BayStars in NPB (Nippon Professional Baseball). He quickly made an impact in his rookie season, posting strong numbers and establishing himself as a reliable arm in the Cubs' rotation.

This notebook examines how Imanaga's pitching style evolved from his impressive **rookie year (2024)** into his **sophomore season (2025)**, where he faced the challenge of MLB hitters having a full year of data to study.

### Analysis Structure

We compare **three periods** to capture the progression:
- **2024**: Full rookie season (29 games, 2,594 pitches)
- **2025 1st Half (1H)**: Pre-All-Star Break (12 games, 1,026 pitches)
- **2025 2nd Half (2H)**: Post-All-Star Break (13 games, 1,152 pitches)

### Topics Covered
- Pitch arsenal and usage evolution
- Velocity and spin rate trends
- Inning-by-inning fatigue patterns
- Whiff rate changes by pitch type
- Two-strike putaway strategies
- Count-based pitch selection
- Batted ball quality metrics
- Time through the order effects
- Left/right batter splits
- Split-finger (FS) location analysis

**Data Source**: [Baseball Savant](https://baseballsavant.mlb.com/) / Statcast via [pybaseball](https://github.com/jldbc/pybaseball)

---

In [ ]:
!pip install duckdb -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# ====== Settings ======
PITCHER_ID = 684007  # Shota Imanaga MLBAM ID
YEARS = [2024, 2025]
GAME_TYPE = 'R'  # Regular season only
ASB_DATE = '2025-07-15'  # All-Star Break cutoff for 1H/2H split
# ======================

## 1. Data Acquisition

We fetch Statcast pitch-level data for Shota Imanaga (MLBAM ID: 684007) for both 2024 and 2025 seasons, then split 2025 into first half and second half using the All-Star Break as the dividing line.

In [ ]:
# Load from Kaggle dataset (much faster than API calls)
df_all = pd.read_csv('/kaggle/input/japanese-mlb-players-statcast/japanese_mlb_pitching.csv')
df_raw = df_all[df_all['pitcher'] == PITCHER_ID].copy()
df_raw['season'] = df_raw['game_year']

for year in YEARS:
    n = len(df_raw[df_raw['season'] == year])
    print(f'  {year}: {n:,} pitches')

print(f'\nTotal (raw): {len(df_raw):,} pitches')

# Filter regular season only
con = duckdb.connect()
df = con.execute(f"""
    SELECT *,
        CASE
            WHEN season = 2024 THEN '2024'
            WHEN season = 2025 AND game_date < '{ASB_DATE}' THEN '2025-1H'
            ELSE '2025-2H'
        END as period
    FROM df_raw
    WHERE game_type = '{GAME_TYPE}'
""").df()

print(f'Total (regular season): {len(df):,} pitches')
print(f'\nPeriod breakdown:')
for period in ['2024', '2025-1H', '2025-2H']:
    n = len(df[df['period'] == period])
    print(f'  {period}: {n:,} pitches')

## 2. Data Summary

A high-level overview of each period: total pitches, games started, average/max velocity, spin rate, and number of distinct pitch types used.

In [ ]:
PERIODS = ['2024', '2025-1H', '2025-2H']

summary = con.execute("""
    SELECT
        period,
        COUNT(*) as pitches,
        COUNT(DISTINCT game_date) as games,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(MAX(release_speed), 1) as max_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin,
        COUNT(DISTINCT pitch_type) as pitch_types
    FROM df
    GROUP BY period
    ORDER BY period
""").df()

print('=== Period Overview ===')
print(summary.to_string(index=False))
print(f'\nTotal: {len(df):,} pitches')

## 3. Pitch Arsenal Overview

Breaking down each pitch type's usage percentage, average velocity, and spin rate across the three periods. This reveals which pitches Imanaga added, dropped, or adjusted.

In [ ]:
arsenal = con.execute("""
    SELECT
        period,
        pitch_type,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY period), 1) as pct,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY period, pitch_type
    ORDER BY period, count DESC
""").df()

print('=== Pitch Arsenal by Period ===')
for period in PERIODS:
    data = arsenal[arsenal['period'] == period]
    print(f'\n--- {period} ---')
    print(data[['pitch_type', 'count', 'pct', 'avg_velo', 'avg_spin']].to_string(index=False))

## 4. Pitch Mix Evolution

Visualizing how pitch selection changed across the three periods. The stacked bar chart makes it easy to spot the dramatic increase in sweeper (ST) usage and the corresponding decrease in split-finger (FS) in the 2025 second half.

In [ ]:
mix_pivot = arsenal.pivot_table(index='period', columns='pitch_type', values='pct', fill_value=0)
mix_pivot = mix_pivot.reindex(PERIODS)

mix_pivot.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='Set3')
plt.title('Shota Imanaga - Pitch Mix by Period')
plt.xlabel('Period')
plt.ylabel('Usage %')
plt.legend(title='Pitch Type', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()

# === Text Summary ===
print('\n=== Pitch Mix (% usage) ===')
print(mix_pivot.round(1).to_string())

# 2024 vs 2025-2H changes
if '2024' in mix_pivot.index and '2025-2H' in mix_pivot.index:
    first = mix_pivot.loc['2024']
    last = mix_pivot.loc['2025-2H']
    diff = (last - first).sort_values()
    print(f'\n=== Biggest Changes (2024 → 2025-2H) ===')
    for pitch, change in diff.items():
        if abs(change) >= 1.0:
            direction = '↑' if change > 0 else '↓'
            print(f'  {pitch}: {first[pitch]:.1f}% → {last[pitch]:.1f}% ({direction}{abs(change):.1f}%)')

## 5. Velocity Trends by Period

Tracking how average velocity and spin rate changed for the top 4 most-used pitches. Velocity decline from year to year could indicate fatigue, mechanical changes, or intentional adjustments.

In [ ]:
velo_by_period = con.execute("""
    SELECT
        period,
        pitch_type,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin,
        COUNT(*) as count
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY period, pitch_type
    ORDER BY period
""").df()

top_pitches = con.execute("""
    SELECT pitch_type FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY pitch_type
    ORDER BY COUNT(*) DESC
    LIMIT 4
""").df()['pitch_type'].tolist()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

period_order = {p: i for i, p in enumerate(PERIODS)}
for pitch in top_pitches:
    data = velo_by_period[velo_by_period['pitch_type'] == pitch].copy()
    data['period_idx'] = data['period'].map(period_order)
    data = data.sort_values('period_idx')
    axes[0].plot(data['period'], data['avg_velo'], marker='o', label=pitch, linewidth=2)
    axes[1].plot(data['period'], data['avg_spin'], marker='o', label=pitch, linewidth=2)

axes[0].set_title('Average Velocity by Period')
axes[0].set_ylabel('Velocity (mph)')
axes[0].legend()

axes[1].set_title('Average Spin Rate by Period')
axes[1].set_ylabel('Spin Rate (rpm)')
axes[1].legend()

plt.suptitle('Shota Imanaga - Velocity & Spin Trends')
plt.tight_layout()
plt.show()

# === Text Summary ===
print('\n=== Velocity & Spin by Period (Top Pitches) ===')
for pitch in top_pitches:
    data = velo_by_period[velo_by_period['pitch_type'] == pitch]
    print(f'\n{pitch}:')
    print(data[['period', 'avg_velo', 'avg_spin', 'count']].to_string(index=False))

## 6. Monthly Velocity Trend (2025 Focus)

A finer-grained look at 2025 velocity by month, compared against 2024. This helps identify whether velocity changes were gradual over the season or happened at a specific point.

In [ ]:
monthly_velo = con.execute("""
    SELECT
        season,
        EXTRACT(MONTH FROM game_date::DATE) as month,
        pitch_type,
        ROUND(AVG(release_speed), 1) as avg_velo,
        COUNT(*) as pitches
    FROM df
    WHERE pitch_type IN (SELECT pitch_type FROM df GROUP BY pitch_type ORDER BY COUNT(*) DESC LIMIT 3)
    GROUP BY season, month, pitch_type
    HAVING COUNT(*) >= 10
    ORDER BY season, month
""").df()

# Plot 2025 monthly trend
fig, ax = plt.subplots(figsize=(12, 6))
df_2025_monthly = monthly_velo[monthly_velo['season'] == 2025]
for pitch in df_2025_monthly['pitch_type'].unique():
    data = df_2025_monthly[df_2025_monthly['pitch_type'] == pitch]
    ax.plot(data['month'], data['avg_velo'], marker='o', label=pitch, linewidth=2)

ax.axvline(x=7, color='gray', linestyle='--', alpha=0.5, label='ASB')
ax.set_xlabel('Month')
ax.set_ylabel('Velocity (mph)')
ax.set_title('Shota Imanaga - 2025 Monthly Velocity Trend')
ax.set_xticks(range(3, 11))
ax.set_xticklabels(['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])
ax.legend()
plt.tight_layout()
plt.show()

# === Text Summary ===
print('=== 2025 Monthly Velocity ===')
for pitch in df_2025_monthly['pitch_type'].unique():
    data = df_2025_monthly[df_2025_monthly['pitch_type'] == pitch]
    print(f'\n{pitch}:')
    print(data[['month', 'avg_velo', 'pitches']].to_string(index=False))

# Also show 2024 for comparison
print('\n=== 2024 Monthly Velocity ===')
df_2024_monthly = monthly_velo[monthly_velo['season'] == 2024]
for pitch in df_2024_monthly['pitch_type'].unique():
    data = df_2024_monthly[df_2024_monthly['pitch_type'] == pitch]
    print(f'\n{pitch}:')
    print(data[['month', 'avg_velo', 'pitches']].to_string(index=False))

## 7. Fatigue Pattern: Inning-by-Inning Velocity

Examining how fastball velocity drops as the game progresses. A steeper decline in later innings could suggest stamina issues or a need for bullpen support.

In [ ]:
ff_type = 'FF' if 'FF' in top_pitches else top_pitches[0]

fatigue = con.execute(f"""
    SELECT
        period,
        inning,
        ROUND(AVG(release_speed), 1) as avg_velo,
        COUNT(*) as pitches
    FROM df
    WHERE pitch_type = '{ff_type}' AND inning <= 8
    GROUP BY period, inning
    HAVING COUNT(*) >= 5
    ORDER BY period, inning
""").df()

fig, ax = plt.subplots(figsize=(12, 6))
for period in PERIODS:
    data = fatigue[fatigue['period'] == period]
    if len(data) > 0:
        ax.plot(data['inning'], data['avg_velo'], marker='o', label=period, linewidth=2)

ax.set_xlabel('Inning')
ax.set_ylabel(f'{ff_type} Velocity (mph)')
ax.set_title(f'Shota Imanaga - {ff_type} Velocity by Inning')
ax.set_xticks(range(1, 9))
ax.legend()
plt.tight_layout()
plt.show()

# === Text Summary ===
print(f'\n=== {ff_type} Velocity by Inning ===')
fatigue_pivot = fatigue.pivot_table(index='inning', columns='period', values='avg_velo')
if len(fatigue_pivot.columns) > 0:
    fatigue_pivot = fatigue_pivot.reindex(columns=PERIODS)
print(fatigue_pivot.to_string())

print(f'\n=== Velocity Drop (1st inning → last inning) ===')
for period in PERIODS:
    data = fatigue[fatigue['period'] == period]
    if len(data) >= 2:
        first_velo = data.iloc[0]['avg_velo']
        last_velo = data.iloc[-1]['avg_velo']
        last_inn = int(data.iloc[-1]['inning'])
        drop = last_velo - first_velo
        print(f'  {period}: {first_velo} → {last_velo} (inn {last_inn}) = {drop:+.1f} mph')

## 8. Whiff Rate Analysis

Whiff rate (swinging strikes / total swings) is a key indicator of a pitch's effectiveness. We track how each pitch type's ability to generate swings and misses changed across the three periods.

In [ ]:
whiff = con.execute("""
    SELECT
        period,
        pitch_type,
        COUNT(*) as total_pitches,
        SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) as whiffs,
        SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END) as total_swings,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) /
        NULLIF(SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END), 0), 1) as whiff_rate
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY period, pitch_type
    ORDER BY period, total_pitches DESC
""").df()

fig, ax = plt.subplots(figsize=(12, 6))
period_order = {p: i for i, p in enumerate(PERIODS)}
for pitch in top_pitches:
    data = whiff[whiff['pitch_type'] == pitch].copy()
    data['period_idx'] = data['period'].map(period_order)
    data = data.sort_values('period_idx')
    if len(data) > 0:
        ax.plot(data['period'], data['whiff_rate'], marker='o', label=pitch, linewidth=2)

ax.set_ylabel('Whiff Rate (%)')
ax.set_title('Shota Imanaga - Whiff Rate by Pitch Type')
ax.legend()
plt.tight_layout()
plt.show()

# === Text Summary ===
print('\n=== Whiff Rate by Pitch Type ===')
whiff_pivot = whiff.pivot_table(index='pitch_type', columns='period', values='whiff_rate')
if len(whiff_pivot.columns) > 0:
    whiff_pivot = whiff_pivot.reindex(columns=PERIODS)
print(whiff_pivot.round(1).to_string())

## 9. Two-Strike Strategy

With two strikes, pitchers shift to their best "putaway" pitches. This section reveals how Imanaga's two-strike approach evolved -- particularly the growing role of the sweeper (ST) as a strikeout weapon.

In [ ]:
two_strike = con.execute("""
    SELECT
        period,
        pitch_type,
        COUNT(*) as pitches,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY period), 1) as pct,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) /
        NULLIF(SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END), 0), 1) as whiff_rate
    FROM df
    WHERE strikes = 2 AND pitch_type IS NOT NULL
    GROUP BY period, pitch_type
    ORDER BY period, pitches DESC
""").df()

print('=== Two-Strike Pitch Selection ===')
for period in PERIODS:
    data = two_strike[two_strike['period'] == period].head(5)
    print(f'\n--- {period} ---')
    print(data[['pitch_type', 'pitches', 'pct', 'whiff_rate']].to_string(index=False))

## 10. Count Situation Analysis

How does Imanaga adjust his pitch selection based on the count? We look at pitch usage when ahead, even, behind, and in full counts to understand his strategic tendencies.

In [ ]:
count_analysis = con.execute("""
    SELECT
        period,
        CASE
            WHEN balls = 3 AND strikes = 2 THEN 'Full Count'
            WHEN balls > strikes THEN 'Behind'
            WHEN strikes > balls THEN 'Ahead'
            ELSE 'Even'
        END as count_situation,
        pitch_type,
        COUNT(*) as pitches,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(
            PARTITION BY period,
            CASE
                WHEN balls = 3 AND strikes = 2 THEN 'Full Count'
                WHEN balls > strikes THEN 'Behind'
                WHEN strikes > balls THEN 'Ahead'
                ELSE 'Even'
            END
        ), 1) as pct
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY period, count_situation, pitch_type
    ORDER BY period, count_situation, pitches DESC
""").df()

print('=== Pitch Selection by Count Situation ===')
for period in PERIODS:
    print(f'\n=== {period} ===')
    for situation in ['Ahead', 'Even', 'Behind', 'Full Count']:
        data = count_analysis[
            (count_analysis['period'] == period) &
            (count_analysis['count_situation'] == situation)
        ].head(3)
        if len(data) > 0:
            top_str = ', '.join([f"{r['pitch_type']} {r['pct']}%" for _, r in data.iterrows()])
            print(f'  {situation}: {top_str}')

## 11. Batted Ball Results

When hitters make contact, how hard are they hitting the ball? We examine exit velocity, launch angle, hard-hit percentage, xBA, and xwOBA across periods to assess whether batters are making better contact over time.

In [ ]:
batted = con.execute("""
    SELECT
        period,
        COUNT(*) as batted_balls,
        ROUND(AVG(launch_speed), 1) as avg_exit_velo,
        ROUND(AVG(launch_angle), 1) as avg_launch_angle,
        ROUND(100.0 * SUM(CASE WHEN launch_speed >= 95 THEN 1 ELSE 0 END) / COUNT(*), 1) as hard_hit_pct,
        ROUND(AVG(estimated_ba_using_speedangle), 3) as avg_xBA,
        ROUND(AVG(estimated_woba_using_speedangle), 3) as avg_xwOBA
    FROM df
    WHERE launch_speed IS NOT NULL
    GROUP BY period
    ORDER BY period
""").df()

print('=== Batted Ball Results by Period ===')
print(batted.to_string(index=False))

# By pitch type
batted_by_pitch = con.execute("""
    SELECT
        period,
        pitch_type,
        COUNT(*) as batted_balls,
        ROUND(AVG(launch_speed), 1) as avg_exit_velo,
        ROUND(AVG(estimated_ba_using_speedangle), 3) as avg_xBA
    FROM df
    WHERE launch_speed IS NOT NULL AND pitch_type IS NOT NULL
    GROUP BY period, pitch_type
    HAVING COUNT(*) >= 10
    ORDER BY period, batted_balls DESC
""").df()

print('\n=== Batted Ball by Pitch Type (min 10 BIP) ===')
for period in PERIODS:
    data = batted_by_pitch[batted_by_pitch['period'] == period]
    print(f'\n--- {period} ---')
    print(data[['pitch_type', 'batted_balls', 'avg_exit_velo', 'avg_xBA']].to_string(index=False))

## 12. Time Through the Order

Do hitters perform better the second or third time they face Imanaga in the same game? This analysis tracks whiff rate and batted ball quality by plate appearance number, revealing how well he sustains effectiveness over multiple trips through the lineup.

In [ ]:
# Time Through Order analysis
# at_bat_number resets per game, we approximate TTO by grouping at_bat_number
tto = con.execute("""
    WITH batter_pa AS (
        SELECT
            period,
            game_pk,
            batter,
            at_bat_number,
            DENSE_RANK() OVER(PARTITION BY game_pk, batter ORDER BY at_bat_number) as pa_num
        FROM df
        GROUP BY period, game_pk, batter, at_bat_number
    ),
    tto_tagged AS (
        SELECT
            d.*,
            CASE
                WHEN b.pa_num = 1 THEN '1st'
                WHEN b.pa_num = 2 THEN '2nd'
                ELSE '3rd+'
            END as tto
        FROM df d
        JOIN batter_pa b ON d.game_pk = b.game_pk AND d.batter = b.batter AND d.at_bat_number = b.at_bat_number
    )
    SELECT
        period,
        tto,
        COUNT(*) as pitches,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) /
        NULLIF(SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END), 0), 1) as whiff_rate
    FROM tto_tagged
    GROUP BY period, tto
    ORDER BY period, tto
""").df()

print('=== Whiff Rate by Time Through Order ===')
tto_pivot = tto.pivot_table(index='tto', columns='period', values='whiff_rate')
tto_pivot = tto_pivot.reindex(columns=PERIODS)
print(tto_pivot.round(1).to_string())

# Batted ball quality by TTO
tto_batted = con.execute("""
    WITH batter_pa AS (
        SELECT
            period,
            game_pk,
            batter,
            at_bat_number,
            DENSE_RANK() OVER(PARTITION BY game_pk, batter ORDER BY at_bat_number) as pa_num
        FROM df
        GROUP BY period, game_pk, batter, at_bat_number
    ),
    tto_tagged AS (
        SELECT
            d.*,
            CASE
                WHEN b.pa_num = 1 THEN '1st'
                WHEN b.pa_num = 2 THEN '2nd'
                ELSE '3rd+'
            END as tto
        FROM df d
        JOIN batter_pa b ON d.game_pk = b.game_pk AND d.batter = b.batter AND d.at_bat_number = b.at_bat_number
    )
    SELECT
        period,
        tto,
        COUNT(*) as batted_balls,
        ROUND(AVG(launch_speed), 1) as avg_exit_velo,
        ROUND(AVG(estimated_woba_using_speedangle), 3) as avg_xwOBA
    FROM tto_tagged
    WHERE launch_speed IS NOT NULL
    GROUP BY period, tto
    ORDER BY period, tto
""").df()

print('\n=== Batted Ball by Time Through Order ===')
for period in PERIODS:
    data = tto_batted[tto_batted['period'] == period]
    print(f'\n--- {period} ---')
    print(data[['tto', 'batted_balls', 'avg_exit_velo', 'avg_xwOBA']].to_string(index=False))

## 13. Left/Right Batter Splits

As a left-handed pitcher, Imanaga has a natural advantage against LHB with his sweeper (ST) breaking away from them. Here we examine how pitch usage and effectiveness differ when facing left-handed vs. right-handed batters.

In [ ]:
# L/R splits - pitch usage and effectiveness
lr_arsenal = con.execute("""
    SELECT
        period,
        stand,
        pitch_type,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY period, stand), 1) as pct,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) /
        NULLIF(SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END), 0), 1) as whiff_rate
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY period, stand, pitch_type
    HAVING COUNT(*) >= 10
    ORDER BY period, stand, count DESC
""").df()

print('=== Pitch Usage & Whiff Rate by Batter Side ===')
for period in PERIODS:
    print(f'\n=== {period} ===')
    for side in ['L', 'R']:
        data = lr_arsenal[(lr_arsenal['period'] == period) & (lr_arsenal['stand'] == side)]
        print(f'\n  vs {side}HB:')
        print(data[['pitch_type', 'count', 'pct', 'whiff_rate']].to_string(index=False))

# Batted ball by side
lr_batted = con.execute("""
    SELECT
        period,
        stand,
        COUNT(*) as batted_balls,
        ROUND(AVG(launch_speed), 1) as avg_exit_velo,
        ROUND(AVG(estimated_woba_using_speedangle), 3) as avg_xwOBA,
        ROUND(100.0 * SUM(CASE WHEN launch_speed >= 95 THEN 1 ELSE 0 END) / COUNT(*), 1) as hard_hit_pct
    FROM df
    WHERE launch_speed IS NOT NULL
    GROUP BY period, stand
    ORDER BY period, stand
""").df()

print('\n=== Batted Ball by Batter Side ===')
print(lr_batted.to_string(index=False))

# ST-specific L/R splits
lr_st = con.execute("""
    SELECT
        period,
        stand,
        COUNT(*) as pitches,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) /
        NULLIF(SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END), 0), 1) as whiff_rate,
        ROUND(AVG(CASE WHEN launch_speed IS NOT NULL THEN estimated_ba_using_speedangle END), 3) as xBA_on_contact
    FROM df
    WHERE pitch_type = 'ST'
    GROUP BY period, stand
    ORDER BY period, stand
""").df()

print('\n=== ST (Sweeper) Left/Right Splits ===')
print(lr_st.to_string(index=False))

## 14. Monthly Batted Ball Metrics

A month-by-month view of batted ball quality for both seasons. This granular breakdown pinpoints exactly when hitters started making harder contact, rather than relying on the coarser 1H/2H split.

In [ ]:
# Monthly batted ball metrics
monthly_batted = con.execute("""
    SELECT
        season,
        EXTRACT(MONTH FROM game_date::DATE) as month,
        COUNT(*) as batted_balls,
        ROUND(AVG(launch_speed), 1) as avg_exit_velo,
        ROUND(100.0 * SUM(CASE WHEN launch_speed >= 95 THEN 1 ELSE 0 END) / COUNT(*), 1) as hard_hit_pct,
        ROUND(AVG(estimated_woba_using_speedangle), 3) as avg_xwOBA
    FROM df
    WHERE launch_speed IS NOT NULL
    GROUP BY season, month
    HAVING COUNT(*) >= 20
    ORDER BY season, month
""").df()

# Plot xwOBA monthly
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for year in YEARS:
    data = monthly_batted[monthly_batted['season'] == year]
    axes[0].plot(data['month'], data['avg_xwOBA'], marker='o', label=str(year), linewidth=2)
    axes[1].plot(data['month'], data['hard_hit_pct'], marker='o', label=str(year), linewidth=2)

axes[0].axvline(x=7, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('xwOBA')
axes[0].set_title('Monthly xwOBA (lower = better)')
axes[0].legend()

axes[1].axvline(x=7, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Hard Hit %')
axes[1].set_title('Monthly Hard Hit % (lower = better)')
axes[1].legend()

plt.suptitle('Shota Imanaga - Monthly Batted Ball Quality')
plt.tight_layout()
plt.show()

# === Text Summary ===
print('=== Monthly Batted Ball Metrics ===')
for year in YEARS:
    data = monthly_batted[monthly_batted['season'] == year]
    print(f'\n--- {year} ---')
    print(data[['month', 'batted_balls', 'avg_exit_velo', 'hard_hit_pct', 'avg_xwOBA']].to_string(index=False))

## 15. FS (Split-finger) Zone Analysis

The split-finger fastball is one of Imanaga's signature pitches. This section analyzes where it lands relative to the strike zone -- is it staying in the chase zone where it generates whiffs, or is it catching too much of the plate?

In [ ]:
# FS zone analysis
# zone 1-9 = strike zone, 11-14 = chase/waste zones
fs_zone = con.execute("""
    SELECT
        period,
        CASE
            WHEN zone BETWEEN 1 AND 9 THEN 'In Zone'
            WHEN zone BETWEEN 11 AND 14 THEN 'Chase'
            ELSE 'Waste'
        END as zone_type,
        COUNT(*) as pitches,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY period), 1) as pct,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked'
        ) THEN 1 ELSE 0 END) /
        NULLIF(SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END), 0), 1) as whiff_rate,
        ROUND(100.0 * SUM(CASE WHEN description IN (
            'swinging_strike', 'swinging_strike_blocked',
            'foul', 'foul_tip', 'foul_bunt',
            'hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score'
        ) THEN 1 ELSE 0 END) / COUNT(*), 1) as swing_rate
    FROM df
    WHERE pitch_type = 'FS' AND zone IS NOT NULL
    GROUP BY period, zone_type
    ORDER BY period, zone_type
""").df()

print('=== FS Zone Analysis ===')
for period in PERIODS:
    data = fs_zone[fs_zone['period'] == period]
    print(f'\n--- {period} ---')
    print(data[['zone_type', 'pitches', 'pct', 'swing_rate', 'whiff_rate']].to_string(index=False))

# FS location scatter by period
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, period in enumerate(PERIODS):
    fs_data = con.execute(f"""
        SELECT plate_x, plate_z, description
        FROM df
        WHERE pitch_type = 'FS' AND period = '{period}'
          AND plate_x IS NOT NULL AND plate_z IS NOT NULL
    """).df()

    whiff_mask = fs_data['description'].isin(['swinging_strike', 'swinging_strike_blocked'])

    axes[i].scatter(fs_data[~whiff_mask]['plate_x'], fs_data[~whiff_mask]['plate_z'],
                    alpha=0.3, s=20, c='gray', label='Other')
    axes[i].scatter(fs_data[whiff_mask]['plate_x'], fs_data[whiff_mask]['plate_z'],
                    alpha=0.7, s=30, c='red', label='Whiff')

    # Strike zone box (approximate)
    axes[i].plot([-0.83, 0.83, 0.83, -0.83, -0.83],
                 [1.5, 1.5, 3.5, 3.5, 1.5], 'k-', linewidth=1)
    axes[i].set_xlim(-2.5, 2.5)
    axes[i].set_ylim(0, 5)
    axes[i].set_title(f'FS Location - {period}')
    axes[i].set_xlabel('Plate X')
    axes[i].set_ylabel('Plate Z')
    axes[i].legend(fontsize=8)
    axes[i].set_aspect('equal')

plt.suptitle('Shota Imanaga - FS (Split-finger) Location by Period')
plt.tight_layout()
plt.show()

# Average FS location
fs_location = con.execute("""
    SELECT
        period,
        ROUND(AVG(plate_x), 2) as avg_plate_x,
        ROUND(AVG(plate_z), 2) as avg_plate_z,
        ROUND(AVG(pfx_x), 1) as avg_h_break,
        ROUND(AVG(pfx_z), 1) as avg_v_break,
        COUNT(*) as pitches
    FROM df
    WHERE pitch_type = 'FS' AND plate_x IS NOT NULL
    GROUP BY period
    ORDER BY period
""").df()

print('\n=== FS Average Location & Movement ===')
print(fs_location.to_string(index=False))

## 16. Summary

A consolidated printout of all key statistics across the three periods.

In [ ]:
print('=' * 60)
print('SHOTA IMANAGA 2024-2025 ANALYSIS SUMMARY')
print('=' * 60)

# Games & Pitches
print('\n[Workload]')
for _, row in summary.iterrows():
    print(f'  {row["period"]}: {int(row["games"])} games, {int(row["pitches"]):,} pitches, avg {row["avg_velo"]} mph')

# Pitch mix changes
print(f'\n[Pitch Mix Changes]')
for period_pair in [('2024', '2025-1H'), ('2025-1H', '2025-2H')]:
    p1, p2 = period_pair
    if p1 in mix_pivot.index and p2 in mix_pivot.index:
        first = mix_pivot.loc[p1]
        last = mix_pivot.loc[p2]
        diff = (last - first).sort_values()
        changes = [(p, c) for p, c in diff.items() if abs(c) >= 2.0]
        if changes:
            print(f'  {p1} → {p2}:')
            for pitch, change in changes:
                direction = '↑' if change > 0 else '↓'
                print(f'    {pitch}: {first[pitch]:.1f}% → {last[pitch]:.1f}% ({direction}{abs(change):.1f}%)')

# Velocity
print(f'\n[Fastball Velocity]')
ff_data = velo_by_period[velo_by_period['pitch_type'] == ff_type]
for _, row in ff_data.iterrows():
    print(f'  {row["period"]}: {row["avg_velo"]} mph ({int(row["count"])} pitches)')

# Best whiff pitch per period
print(f'\n[Best Whiff Rate Pitch]')
for period in PERIODS:
    period_whiff = whiff[(whiff['period'] == period) & (whiff['total_swings'] >= 20)]
    if len(period_whiff) > 0:
        best = period_whiff.loc[period_whiff['whiff_rate'].idxmax()]
        print(f'  {period}: {best["pitch_type"]} ({best["whiff_rate"]}%)')

# Batted ball
print(f'\n[Batted Ball Quality]')
for _, row in batted.iterrows():
    print(f'  {row["period"]}: xwOBA {row["avg_xwOBA"]}, Hard Hit {row["hard_hit_pct"]}%, Exit Velo {row["avg_exit_velo"]} mph')

print('\n' + '=' * 60)

## Key Findings & Conclusion

### Pitch Arsenal Foundation
Imanaga's four-seam fastball (FF) and split-finger (FS) together account for roughly **80% of all pitches** across every period. This FF+FS combination is the foundation of his game -- the fastball sets up the split, and the split keeps hitters off the fastball.

### The Sweeper Revolution
The most dramatic change in Imanaga's arsenal is the **sweeper (ST) usage surge** in the 2025 second half:
- 2024: 7.6% usage
- 2025 1H: 11.1% usage
- 2025 2H: **21.5% usage** (nearly tripled from 2024)

The sweeper is particularly devastating against left-handed batters, generating a **42% whiff rate** vs LHB consistently across all three periods. In the 2025 2H, Imanaga began deploying it aggressively against right-handed batters as well (18.1% usage vs RHB, up from 1.1% in 2024).

### Batters Adjusting
Despite the sweeper evolution, the 2025 second half showed signs of **batters catching up**:
- **Hard hit rate** increased from 22.7% (2024) to 28.8% (2025 2H)
- **xwOBA** rose from .366 (2024) to .408 (2025 2H)
- Exit velocity on fastballs climbed from 83.4 to 84.5 mph

### Velocity Decline
Fastball velocity showed a modest but consistent decline:
- 2024: 91.7 mph average
- 2025 1H: 90.9 mph
- 2025 2H: 90.8 mph

The max velocity also dropped from 94.6 to 93.0 mph, suggesting this is not just a selective usage change but a genuine velocity reduction.

### Key to Watch
The central question for Imanaga going forward is whether the **increased sweeper usage can offset batters' adjustments** to his FF+FS core. The sweeper gives him a legitimate third weapon with a different movement profile, but the rising hard-hit rate and xwOBA in 2025 2H suggest hitters are beginning to square up his pitches more effectively. His ability to continue evolving his pitch mix -- and potentially regain some velocity -- will determine whether he remains a frontline starter or settles into a mid-rotation role.

---

*Data: Baseball Savant / Statcast via pybaseball. Analysis covers regular season only.*